Input/output
============

Requirements
------------



This libraries have to be installed on your environment.

In [ ]:
import h5py
import silx
import fabio

EDF using FabIO
===============

For simple access, or fast access to raster format like EDF, MSK, TIFF... we recommand to use fabio `fabio`.

Reading files
-------------

In [ ]:
import fabio

image = fabio.open("data/medipix.edf")

# here is the data as a numpy array
image.data
print("Image size:", image.data.shape)

# here is the header as key-value dictionary
image.header
print("Metadata count:", len(image.header))
print("Metadata names:", list(image.header.keys()))

None

Play with it. You can see, it is easy to access to the image data, it is easy to access to simple metadata like *pixel size*. It is more difficult to access to motors or counters.

Writing files
-------------

In [ ]:
import fabio
import numpy

image = numpy.random.rand(10, 10)
metadata = {'pixel_size': '0.2'}

image = fabio.edfimage.edfimage(data=image, header=metadata)
image.write('new.edf')

Convert files
---------------------

In [ ]:
import fabio

image = fabio.open('data/medipix.edf')
image = image.convert('tif')
image.save('filename.tif')

In [ ]:
ls -al filename.*

HDF5 using h5py
===============

Read example
------------

In [ ]:
import h5py

h5file = h5py.File('data/test.h5')

# print available names at the first level
print("First children:", list(h5file['/'].keys()))

# reaching a dataset from a sub group
dataset = h5file['/diff_map_0000/data/map']

# using size and types to not read the full stored data
print("Dataset:", dataset.shape, dataset.size, dataset.dtype)

# datasets mimics numpy-array
# read and apply the operation
a = 2 * dataset[0, 5]
# copy the data and store it as a numpy-array
b = dataset[...]

Write example
-------------

In [ ]:
import numpy
import h5py

data = numpy.arange(10000.0)
data.shape = 100, 100

# write
h5file = h5py.File('my_first_one.h5', mode='w')

# write data into a dataset from the root
h5file['/data1'] = data

# write data into a dataset from group1
h5file['/group1/data2'] = data

h5file.close()

Using silx io
=============

`silx.io` provides a common API to read spec files, EDF, TIFF, and h5py files. This API is a read-only `h5py`-like.

Tool
-----------------

`h5ls` allow you to display the tree contained into an HDF5 file.

In [ ]:
import silx.io
import silx.io.utils

h5file = silx.io.open('data/test.h5')

string = silx.io.utils.h5ls(h5file)
print(string)

Read spec files using silx
--------------------------

In [ ]:
import silx.io
data = silx.io.open('data/oleg.dat')

# print available scans
print("First childs:", data['/'].keys())

# print available measurements from the scan 94.1
print("Containt of measurement:", data['/94.1/measurement'].keys())

# get data from measurement
xdata = data['/94.1/measurement/Epoch']
ydata = data['/94.1/measurement/bpmi']
for row in zip(xdata, ydata):
    print(row)

Convert spec file to HDF5
-------------------------

In [ ]:
from silx.io.convert import write_to_h5

write_to_h5('data/oleg.dat', 'oleg.h5', mode='w')

In [ ]:
ls -al oleg.*

Read EDF file using silx
------------------------

In [ ]:
import silx.io
data = silx.io.open('data/ID16B_diatomee.edf')

# Access to the frames
frames = data['/scan_0/instrument/detector_0/data']
len(frames)  # number of frames
frames[0]    # first frame
print("Number of frames:", len(frames))
print("Size of an image:", frames[0].shape)

# Access to motors, monitor, timestanp
srot = data['scan_0/instrument/positioners/srot'][...]
mon = data['scan_0/measurement/mon'][...]
timestamp = data['scan_0/instrument/detector_0/others/time_of_day'][...]
for row in zip(timestamp, srot, mon):
    print(row)

Read HDF5 using silx
--------------------

For conveniance, ``silx`` also provides the h5py API for HDF5 files.

In [ ]:
import silx.io

h5file = silx.io.open('data/test.h5')

# print available names at the first level
print("First children:", list(h5file['/'].keys()))

# reaching a dataset from a sub group
dataset = h5file['/diff_map_0000/data/map']

# using size and types to not read the full stored data
print("Dataset:", dataset.shape, dataset.size, dataset.dtype)

# datasets mimics numpy-array
# read and apply the operation
a = 2 * dataset[0, 5]
# copy the data and store it as a numpy-array
b = dataset[...]

Exercise
========

1. Read the EDF file ``medipix.edf``.
2. Process the data
   The goal of the processing is to clamp the pixels values to a new range of values ([10%, 90%] of the existing one). To do so:

   - Create a mask to detect pixel which are below 10% or above 90% of the current range.
   - With the above mask, set the affected pixels to 10% 'low value'.

3. Store the source, the mask of changed pixels and the result inside ``process.h5``, as below.

   ![Output file structure](images/exercise-result.png)

4. Load ``process.h5`` and list the root content


In [ ]:
# Load data/medipix.edf
# ...

# Process the data
# ...

# Save data into a new file (process.h5)
# ...

# Load process.h5 and list the root content
# ...

Solution
========

In [ ]:
# Load data/medipix.edf
import exercicesolution
import inspect
print(inspect.getsource(exercicesolution.load_data))

In [ ]:
# process data
import exercicesolution
import inspect
print(inspect.getsource(exercicesolution.process_data))

In [ ]:
# save data
import exercicesolution
import inspect
print(inspect.getsource(exercicesolution.save_data))

In [ ]:
# list root
import exercicesolution
import inspect
print(inspect.getsource(exercicesolution.list_root))

In [1]:
# result
import exercicesolution
import inspect
raw_data, proc_data, mask = exercicesolution.solution("data/medipix.edf")

root level:
['mask', 'raw', 'result']


In [ ]:
%pylab

In [ ]:
imshow(mask)

In [ ]:
imshow(raw_data)

In [ ]:
imshow(proc_data)